Import Packages


In [ ]:

import os
import time
import math
import string
import random

import collections
import numpy as np
import matplotlib.pyplot as plt

import torch
from datasets import load_dataset
from torch import nn
from torch import optim
from torch.functional import F
from torch.utils.data import DataLoader



# Data

In [ ]:
def keys_to_values(keys, map, default_if_missing=None):
    return [map.get(key, default_if_missing) for key in keys]

In [13]:
seed = 42
torch.manual_seed(seed)
# Probably, this below must be changed if you work with a M1/M2/M3 Mac
torch.cuda.manual_seed(seed) # for CUDA
torch.backends.cudnn.deterministic = True # for CUDNN
torch.backends.benchmark = False # if True, causes cuDNN to benchmark multiple convolution algorithms and select the fastest.

# Q1

In [14]:
# Question 1
ds = load_dataset("heegyu/news-category-dataset")
print(ds['train'])
print(ds['train'].column_names)


Dataset({
    features: ['link', 'headline', 'category', 'short_description', 'authors', 'date'],
    num_rows: 209527
})
['link', 'headline', 'category', 'short_description', 'authors', 'date']


i. dataset is a package providing two main features: the first one is to be a dataloader for many public datasets, and the second one is a way to be efficient in the data pre-processing


ii. The dataset is loaded as an object of the DatasetDict type, provided by Hugging Face Datasets. It is essentially a Python dictionary where: Keys represent dataset splits (e.g., train, test, validation). Values are Dataset objects, which are table-like structures similar to pandas DataFrames or NumPy arrays.


iii. ['link', 'headline', 'category', 'short_description', 'authors', 'date'], in total 6, the first 5 in string and the last one is in s, are fundamental for nlp processing

# Q2

In [15]:
ds = ds['train'].filter(lambda x: x['category'] == 'POLITICS')
print(len(ds))


35602


# Q3

In [ ]:
tokenized_text = []
for title in ds['headline']:  
    words = title.lower().split()
    words.append('<EOS>')
    tokenized_text.append(words)
    
tokenized_text[:2]

[['biden',
  'says',
  'u.s.',
  'forces',
  'would',
  'defend',
  'taiwan',
  'if',
  'china',
  'invaded',
  '<EOS>'],
 ['‘beautiful',
  'and',
  'sad',
  'at',
  'the',
  'same',
  'time’:',
  'ukrainian',
  'cultural',
  'festival',
  'takes',
  'on',
  'a',
  'deeper',
  'meaning',
  'this',
  'year',
  '<EOS>'],
 ['biden',
  'says',
  "queen's",
  'death',
  'left',
  "'giant",
  "hole'",
  'for',
  'royal',
  'family',
  '<EOS>'],
 ['bill',
  'to',
  'help',
  'afghans',
  'who',
  'escaped',
  'taliban',
  'faces',
  'long',
  'odds',
  'in',
  'the',
  'senate',
  '<EOS>'],
 ['mark',
  'meadows',
  'complies',
  'with',
  'justice',
  'dept.',
  'subpoena:',
  'report',
  '<EOS>'],
 ['democrats',
  'nominate',
  'seth',
  'magaziner',
  'in',
  'key',
  'rhode',
  'island',
  'house',
  'race',
  '<EOS>'],
 ['joe',
  'biden',
  'urges',
  'national',
  'unity',
  'in',
  'speech',
  'on',
  'renewed',
  '‘cancer',
  'moonshot’',
  '<EOS>'],
 ['sen.',
  'tim',
  'scott',
  'do

#  Q4

In [17]:

word_freq = {}
for sequence in tokenized_text:
    for word in sequence:
        word_freq[word] = word_freq.get(word, 0) + 1

# Sort words by frequency
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

# Create word to int mapping with special tokens at start/end
word_to_int = {'<EOS>': 0}  # EOS token at index 0
current_idx = 1

for word, freq in sorted_words:
    if word != '<EOS>':  # Skip EOS since already added
        word_to_int[word] = current_idx
        current_idx += 1
        
word_to_int['PAD'] = len(word_to_int)  # PAD token at end

# Create reverse mapping
int_to_word = {i: word for word, i in word_to_int.items()}

# Get 5 most common words (no PAD or EO5)
top_5_words = [(word, freq) for word, freq in sorted_words[:5] if word not in ['<EOS>', 'PAD']]

print("5 most common words:")
for word, freq in top_5_words:
    print(f"{word}: {freq} occurrences")

print(f"\nTotal vocabulary size (including special tokens): {len(word_to_int)}")  

5 most common words:
to: 10701 occurrences
the: 9619 occurrences
trump: 6896 occurrences
of: 5536 occurrences

Total vocabulary size (including special tokens): 33207


# Q5


In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class NewsSDataset(Dataset):
    def __init__(self, tokenized_sequences, word_to_int):
        """
        Args:
            tokenized_sequences: List of word sequences
            word_to_int: Dictionary mapping words to integers
        """
        self.sequences = []
        
        # Convert words to integers and store
        for seq in tokenized_sequences:
            int_seq = [word_to_int.get(word, word_to_int['<EOS>']) for word in seq]
            self.sequences.append(int_seq)
            
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        """Returns (x, y) where:
           x = sequence except last word
           y = sequence except first word
        """
        sequence = self.sequences[idx]
        return (
            torch.tensor(sequence[:-1]),  # all but not  the last
            torch.tensor(sequence[1:])    # all but not  the first
        )

def collate_fn(batch, pad_value):
    """
    Pads sequences in batch to same length
    Args:
        batch: List of (x, y) tuples from dataset
        pad_value: Integer index for PAD token
    """
    # Separate xs and ys
    x_seqs, y_seqs = zip(*batch)
    
    # Pad sequences to max length in batch
    x_padded = pad_sequence(x_seqs, batch_first=True, padding_value=pad_value)
    y_padded = pad_sequence(y_seqs, batch_first=True, padding_value=pad_value)
    
    return x_padded, y_padded



# Q6

In [ ]:
batch_size = 8
dataset = NewsSDataset(tokenized_text, word_to_int)
if batch_size == 1:
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
else:
  dataloader = DataLoader(dataset, batch_size=batch_size,
                          collate_fn=lambda b: collate_fn(b, word_to_int["PAD"]),
                          shuffle=True)




# Q7


In [ ]:

class Model(nn.Module):
    def __init__(self, map, hidden_size=1024, emb_dim=150, n_layers=1):
        super(Model, self).__init__()

        self.vocab_size  = len(map)
        self.hidden_size = hidden_size
        self.emb_dim     = emb_dim
        self.n_layers    = n_layers

        # dimensions: batches x seq_length x emb_dim
        self.embedding = nn.Embedding(
            num_embeddings=self.vocab_size,
            embedding_dim =self.emb_dim,
            padding_idx=map["PAD"])

        self.lstm = nn.LSTM(
                    input_size =self.emb_dim,
                    hidden_size=self.hidden_size,
                    num_layers =self.n_layers,
                    batch_first=True)
                
    
        self.fc = nn.Linear(
            in_features =self.hidden_size,
            out_features=self.vocab_size)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        yhat, state = self.lstm(embed, prev_state)   # yhat is the full sequence prediction, while state is the last hidden state (coincides with yhat[-1] if n_layers=1)

        out = self.fc(yhat)
        return out, state

    def init_state(self, b_size=1):
        return (torch.zeros(self.n_layers, b_size, self.hidden_size),
                torch.zeros(self.n_layers, b_size, self.hidden_size))






# Q8


In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'mps' 
    if torch.backends.mps.is_available() else 'cpu')

model = Model(word_to_int).to(DEVICE)

In [ ]:
def random_sample_next(model, x, prev_state, topk=5, uniform=True):
    # Perform forward-prop and get the output of the last time-step
    out, state = model(x.to(DEVICE), prev_state=tuple(s.to(DEVICE) for s in prev_state))
    last_out = out[0, -1, :]    # vocabulary values of last element of sequence
    
    # Get the top-k indexes and their values
    topk = topk if topk else last_out.shape[0]
    top_logit, top_ix = torch.topk(last_out, k=topk, dim=-1)
    
    # Get the softmax of the topk's and sample
    p = None if uniform else F.softmax(top_logit, dim=-1).cpu().detach().numpy()
    
    # top_ix deve essere su CPU per numpy.random.choice
    sampled_ix = np.random.choice(top_ix.cpu().numpy(), p=p)
    
    return sampled_ix, state

In [ ]:
def sample_argmax(model, x, prev_state):
    # Perform forward-prop and get the output of the last time-step
    out, state = model(x.to(DEVICE), prev_state=tuple(s.to(DEVICE) for s in prev_state))
    # Get last timestep prediction and find highest probability word
    last_pred = out[:, -1, :]  # shape: [batch_size, vocab_size]
    next_word = torch.argmax(last_pred, dim=-1).item()
    
    return next_word, state

In [ ]:
def sample(model, prompt,stop_on, sample_method='random', topk=5, max_seqlen=18):
    prompt = prompt if isinstance(prompt, (list, tuple)) else [prompt]

    model.eval()
    with torch.no_grad():
        sampled_ix_list = prompt[:]
        x = torch.tensor([prompt])
        prev_state = model.init_state(b_size=1)
        
        for t in range(max_seqlen - len(prompt)):
            if sample_method == 'argmax':
                # For argmax, set topk=1 and uniform=False
                sampled_ix, prev_state = sample_argmax(model, x, prev_state)
            else:
                # For random, use provided topk and uniform sampling
                sampled_ix, prev_state = random_sample_next(model, x, prev_state, topk=topk)

            sampled_ix_list.append(sampled_ix)
            x = torch.tensor([[sampled_ix]])

            
            
            if sampled_ix == stop_on:
                break

    model.train()
    return sampled_ix_list

In [ ]:
prompt_text = "the president wants"
prompt_indices = keys_to_values(prompt_text.split(), word_to_int)

In [ ]:
print("\nPrompt iniziale:", prompt_text)

# 3. Test con sampling random
print("\nGenerazione con sampling casuale (random):")
for i in range(3):
    # Genera sequenza
    generated_indices = sample(model, prompt_indices,word_to_int['<EOS>'], sample_method='random', topk=5)
    # Converti indici in parole usando keys_to_values
    generated_text = " ".join(keys_to_values(generated_indices, int_to_word))
    print(f"Sequenza {i+1}: {generated_text}")

# 4. Test con strategia greedy
print("\nGenerazione con strategia greedy (argmax):")
for i in range(3):
    generated_indices = sample(model, prompt_indices,word_to_int['<EOS>'],sample_method='argmax')
    generated_text = " ".join(keys_to_values(generated_indices, int_to_word))
    print(f"Sequenza {i+1}: {generated_text}")

# Training


In [ ]:
def train(model, data, num_epochs, criterion, lr=0.001, print_every=50, clip=None):
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(DEVICE)
    criterion = criterion.to(DEVICE)
    model.train()
    
    costs = []
    running_loss = 0
    loss_hist = []
    generated_text_list = []
    perplexity_hist = []
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    total_batches = len(data)
    epoch = 0
    while epoch < num_epochs:
        epoch += 1
        epoch_loss = 0.0
        
        for batch_idx, (x, y) in enumerate(data, 1):
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            optimizer.zero_grad()
            
            # Initialize hidden state
            prev_state = model.init_state(b_size=x.shape[0])
            prev_state = tuple(s.to(DEVICE) for s in prev_state)
            
            # Forward pass
            out, state = model(x, prev_state=prev_state)
            
            # Reshape output for CrossEntropyLoss [batch_size, vocab_size, sequence_length]
            loss_out = out.permute(0, 2, 1)
            
            # Calculate loss
            loss = criterion(loss_out, y)
            epoch_loss += loss.item()
            costs.append(loss.item())
            running_loss += loss.item()
            
            # Backward pass and optimization
            loss.backward()
            if clip:
                nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
        
        # Calculate average loss for the epoch
        avg_epoch_loss = epoch_loss / total_batches
        loss_hist.append(avg_epoch_loss)
        
        # Calculate perplexity directly from cross-entropy loss
        perplexity = torch.exp(torch.tensor(avg_epoch_loss))
        perplexity_hist.append(perplexity.item())
        
        if print_every and (epoch % print_every) == 0:
            print(f"Epoch: {epoch}/{num_epochs}, Loss: {avg_epoch_loss:8.4f}, Perplexity: {perplexity:8.4f}")
            generated_indices = sample(model, prompt_indices, word_to_int['<EOS>'], sample_method='argmax')
            generated_text = " ".join(keys_to_values(generated_indices, int_to_word))
            generated_text_list.append(generated_text)
            print(f"Generated text: {generated_text}\n")
            
        # Early stopping check
        if avg_epoch_loss < 1.5:
            print(f"\nTarget loss of 1.5 reached at epoch {epoch}!")
            break
        
    if len(generated_text_list) > 0:
        print("Beginning list:", generated_text_list[0])
        middle_index = len(generated_text_list) // 2
        print("Middle list:", generated_text_list[middle_index])
        print("End list:", generated_text_list[-1])
        
    return model, costs, loss_hist, perplexity_hist

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=word_to_int["PAD"])
model, costs, loss_hist, perplexity_hist = train(model, dataloader, 12, criterion, lr=1e-3,
                                 print_every=1, clip=1)

In [ ]:
# Plot the training metrics at the end
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(loss_hist, label='Training Loss')
plt.axhline(y=1.5, color='r', linestyle='--', label='Target Loss (1.5)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.grid(True)

# Perplexity plot
plt.subplot(1, 2, 2)
plt.plot(perplexity_hist, label='Perplexity')
plt.xlabel('Epoch')
plt.ylabel('Perplexity')
plt.title('Perplexity Over Time')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Bonus

In [ ]:
# https://blog.dataiku.com/arithmetic-properties-of-word-embeddings